# L3a: Linear regression models for prediction and classification tasks
This lecture explores linear regression models for prediction and classification. Linear regression predicts continuous target variables using one or more features (prediction) and classifies objects based on features (classification). These models are easy to implement and interpret, serving as a foundation for more complex predictive analytics algorithms.

There are several key ideas in this lecture:

* __Linear regression__ is a statistical method for modeling the relationship between a dependent variable (target) and one or more independent variables (features) by fitting a linear equation to observed data. It provides a simple way to predict outcomes and understand relationships between variables.
* __Continuous variable__ prediction tasks: In machine learning, linear regression models are commonly employed for continuous variable prediction tasks. These models enable the estimation of numerical outcomes based on the (non)linear relationships identified between input features and the target variable.
* __Classification tasks__: While linear regression is primarily designed to predict continuous outcomes, it can also be adapted for classification tasks by combining the linear regression model with an output function that transforms the continuous target variable predictions into discrete classes or probability estimates. In this lecture, we'll consider binary classification using [the perceptron classification algorithm, developed at Cornell in the 1950s](https://en.wikipedia.org/wiki/Perceptron).

### Setup, Data, and Prerequisites
We set up the computational environment by including the `Include.jl` file, loading any needed resources, such as sample datasets, and setting up any required constants. The `Include.jl` file loads external packages, various functions that we will use in the exercise, and custom types to model the components of our problem.

In [1]:
include("Include.jl")

  Activating project at `~/Documents/GitHub/CHEME-5820-Lectures-Spring-2025/lectures/week-3/L3a`
    Updating `~/Documents/GitHub/CHEME-5820-Lectures-Spring-2025/lectures/week-3/L3a/Project.toml`
  [336ed68f] + CSV v0.10.15
  [a93c6f00] + DataFrames v1.7.0
  [31c24e10] + Distributions v0.25.117
  [5789e2e9] + FileIO v1.16.6
  [91a5bcdd] + Plots v1.40.9
  [08abe8d2] + PrettyTables v2.4.0
  [10745b16] + Statistics v1.11.1
  [2913bbd2] + StatsBase v0.34.4
  [f3b207a7] + StatsPlots v0.15.7
  [37e2e46d] ~ LinearAlgebra ⇒ v1.11.0
    Updating `~/Documents/GitHub/CHEME-5820-Lectures-Spring-2025/lectures/week-3/L3a/Manifest.toml`
  [621f4979] + AbstractFFTs v1.5.0
  [79e6a3ab] + Adapt v4.1.1
  [66dad0bd] + AliasTables v1.1.3
  [7d9fca2a] + Arpack v0.5.4
  [13072b0f] + AxisAlgorithms v1.1.0
  [d1d4a3ce] + BitFlags v0.1.9
  [336ed68f] + CSV v0.10.15
  [d360d2e6] + ChainRulesCore v1.25.1
  [aaaa29a8] + Clustering v0.15.8
  [944b1d66] + CodecZlib v0.7.6
  [35d6a980] + ColorSchemes v3.28.0
  [3da00

LoadError: LoadError: ArgumentError: Package Colors not found in current path, maybe you meant `import/using .Colors`.
- Otherwise, run `import Pkg; Pkg.add("Colors")` to install the Colors package.
in expression starting at /Users/clarasachmann/Documents/GitHub/CHEME-5820-Lectures-Spring-2025/lectures/week-3/L3a/Include.jl:16

#### Boston housing data

In [2]:
df_bostonhousingdata = CSV.read(joinpath(_PATH_TO_DATA, "data-boston-housing.csv"), DataFrame)

LoadError: UndefVarError: `CSV` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [3]:
D_housing = Matrix(df_bostonhousingdata); # get the data as a Matrix (alias for Array{Float64,2})
number_of_training_examples_housing = 390; # how many training points for the housing dataset?

LoadError: UndefVarError: `df_bostonhousingdata` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Split the housing dataset into training and testing datasets

In [4]:
house_training, house_test = let

    number_of_features = size(D_housing,2); # number of cols of housing data
    number_of_examples = size(D_housing,1); # number of rows of housing data
    full_index_set = range(1,stop=number_of_examples,step=1) |> collect |> Set;
    
    # build index sets for training and testing
    training_index_set = Set{Int64}();
    should_stop_loop = false;
    while (should_stop_loop == false)
        i = rand(1:number_of_examples);
        push!(training_index_set,i);

        if (length(training_index_set) == number_of_training_examples_housing)
            should_stop_loop = true;
        end
    end
    test_index_set = setdiff(full_index_set,training_index_set);

    # build the test and train datasets -
    house_training = D_housing[training_index_set |> collect,:];
    house_test = D_housing[test_index_set |> collect,:];

    # return
    house_training,house_test
end;

LoadError: UndefVarError: `D_housing` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [5]:
house_training

LoadError: UndefVarError: `house_training` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

#### Banknote Authentication Dataset
The second dataset we will explore is the [banknote authentication dataset from the UCI archive](https://archive.ics.uci.edu/dataset/267/banknote+authentication). This dataset has 1372 instances of 4 continuous features and an integer $\{-1,1\}$ class indicator variable. 
* __Description__: Data were extracted from images taken from genuine and forged banknote-like specimens.  An industrial camera, usually used for print inspection, was used for digitization. The final images have 400x 400 pixels. Due to the object lens and distance to the investigated object, gray-scale pictures with a resolution of about 660 dpi were gained. Wavelet Transform tools were used to extract features from images.
* __Features__: The data has four continuous features from each image: `variance` of the wavelet transformed image, `skewness` of the wavelet transformed image, `kurtosis` of the wavelet transformed image, and the `entropy` of the wavelet transformed image. The class is $\{-1,1\}$ where a class value of `-1` indicates genuine, `1` forged.

In [6]:
df_banknote = CSV.read(joinpath(_PATH_TO_DATA, "data-banknote-authentication.csv"), DataFrame)

LoadError: UndefVarError: `CSV` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [7]:
D_banknote = Matrix(df_banknote); # get the data as a Matrix (alias for Array{Float64,2})
number_of_training_examples_banknote = 1200; # how many training points for the banknote dataset?

LoadError: UndefVarError: `df_banknote` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [8]:
banknote_training, banknote_test = let

    number_of_features = size(D_banknote,2); # number of cols of housing data
    number_of_examples = size(D_banknote,1); # number of rows of housing data
    full_index_set = range(1,stop=number_of_examples,step=1) |> collect |> Set;
    
    # build index sets for training and testing
    training_index_set = Set{Int64}();
    should_stop_loop = false;
    while (should_stop_loop == false)
        i = rand(1:number_of_examples);
        push!(training_index_set,i);

        if (length(training_index_set) == number_of_training_examples_housing)
            should_stop_loop = true;
        end
    end
    test_index_set = setdiff(full_index_set,training_index_set);

    # build the test and train datasets -
    banknote_training = D_banknote[training_index_set |> collect,:];
    banknote_test = D_banknote[test_index_set |> collect,:];

    # return
    banknote_training,banknote_test
end;

LoadError: UndefVarError: `D_banknote` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [9]:
banknote_training

LoadError: UndefVarError: `banknote_training` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

## Linear regression models for continuous prediction tasks
Suppose there exists a dataset $\mathcal{D} = \left\{\mathbf{x}_{i},y_{i}\right\}_{i=1}^{n}$ with $n$-training (labeled) examples, where $\mathbf{x}_{i}\in\mathbb{R}^{p}$ is a $p$-vector of features (independent variables, typically real but potentially complex as well) and $y_{i}\in\mathbb{R}$ denotes a scalar response variable (dependent variable). Then, a $\texttt{linear regression model}$ for the dataset $\mathcal{D}$ takes the form:
$$
\begin{equation*}
y_{i} = \mathbf{x}_{i}^{T}\cdot\mathbf{\beta} + \epsilon_{i}\qquad{i=1,2,\dots,n}
\end{equation*}
$$
where $\mathbf{\beta}\in\mathbb{R}^{p}$ is a $p\times{1}$ vector of unknown model parameters, and $\epsilon_{i}\in\mathbb{R}$ is the unobserved random error for response $i$. The linear regression model in matrix-vector form is given by:
$$
\begin{equation*}
\mathbf{y} = \mathbf{X}\cdot\mathbf{\beta} + \mathbf{\epsilon}
\end{equation*}
$$
where $\mathbf{X}$ is an $n\times{p}$ matrix with the features $\mathbf{x}_{i}^{T}$ on the rows, 
the response vector $\mathbf{y}$ is an $n\times{1}$ vector with entries $y_{i}$, 
and the error vector $\mathbf{\epsilon}$ is an $n\times{1}$ vector with entries $\epsilon_{i}$. The challenge of linear regression is to estimate the unknown parameters $\mathbf{\beta}$ from the dataset $\mathcal{D}$.

### Case I: Overdetermined data matrix
Let the data matrix $\mathbf{X}$ be $\texttt{overdetermined}$, i.e., $n > p$ (more rows than columns), and the error vector $\mathbf{\epsilon}\sim\mathcal{N}(\mathbf{0},\sigma^{2}\cdot\mathbf{I})$.
Then, the [ordinary least squares](https://en.wikipedia.org/wiki/Ordinary_least_squares) estimate of the unknown parameters $\mathbf{\beta}$ will $\textit{minimize}$ the sum of squared errors between model estimates and observed values:
$$
\begin{equation*}
\hat{\mathbf{\beta}} = \arg\min_{\mathbf{\beta}} ||~\mathbf{y} - \mathbf{X}\cdot\mathbf{\beta}~||^{2}_{2}
\end{equation*}
$$
where $||\star||^{2}_{2}$ is the square of the [`p = 2` vector norm](https://en.wikipedia.org/wiki/Norm_(mathematics)#Euclidean_norm), and $\hat{\mathbf{\beta}}$ denotes the estimated parameter vector.  The parameters $\hat{\mathbf{\beta}}$ that minimize the $||\star||^{2}_{2}$ loss for an overdetermined data matrix $\mathbf{X}$ are given by:
\begin{equation*}
\hat{\mathbf{\beta}} = \left(\mathbf{X}^{T}\mathbf{X}\right)^{-1}\mathbf{X}^{T}\mathbf{y} - \left(\mathbf{X}^{T}\mathbf{X}\right)^{-1}\mathbf{X}^{T}\mathbf{\epsilon}
\end{equation*}
The matrix $\mathbf{X}^{T}\mathbf{X}$ is the $\texttt{normal matrix}$, while $\mathbf{X}^{T}\mathbf{y}$ is the $\texttt{moment vector}$. The inverse $\left(\mathbf{X}^{T}\mathbf{X}\right)^{-1}$ must exist to obtain the estimated model parameter vectors $\hat{\mathbf{\beta}}$.

In [10]:
synthetic_experimental_data, μ, K = let

    # Chymotrypsin actual parameters (only the oracle knows these), and experimental setup
    number_of_substrate_samples = 100; # how many substrate samples will we measure?
    number_of_enzyme_samples = 10; # how many enzyme concentrations will we measure?
    number_of_replicates = 10; # how many estimates do we take per condition?
    μ = 0.14*60; # units 1/min
    K = 15.0; # units mM
    E = range(0.01, stop=0.1,length=number_of_enzyme_samples) |> collect; # true: set of enzyme concentrations we'll explore (units: mM)
    S = range(0.1, stop=3*K, length = number_of_substrate_samples) |> collect; # true: set of substate concentrations we'll explore (units: mM)
    σ₁ = 0.001*K; # uncertainty in the substrate measurement
    σ₂ = 0.01*μ; # uncertainty in the rate measurement
    σ₃ = 0.001*mean(E); # uncertainty in the enzyme measurement

    d₁ = Normal(0,σ₁); # measurement error substrate
    d₂ = Normal(0,σ₂); # measurement error rate
    d₃ = Normal(0,σ₃); # measurement error enzyme
    
    data = Array{Float64,3}(undef, number_of_substrate_samples*number_of_replicates, 3, number_of_enzyme_samples);

    for i ∈ eachindex(E)
        Ê = E[i]*(1+rand(d₃)); # what we *actually* used in the experiment (diff than what we thought by the error model)
        rowindex = 1;
        for j ∈ eachindex(S)
            Ŝ = S[j]*(1+rand(d₁));
            for k ∈ 1:number_of_replicates
                r = (μ*Ê)*(Ŝ/(K+Ŝ))*(1+rand(d₂));                 
                data[rowindex,1,i] = S[j]; # we record what we *think* should be there
                data[rowindex,2,i] = E[i]; # we record what we *think* should be there
                data[rowindex,3,i] = r; # measurement

                # update -
                rowindex += 1;
            end
        end
    end
    
    data, μ, K; # return
end;

LoadError: UndefVarError: `Normal` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [11]:
let
    scatter(synthetic_experimental_data[:,1,10],synthetic_experimental_data[:,3,10], c=:red, label="E = $(synthetic_experimental_data[1,2,10]) mM")
    scatter!(synthetic_experimental_data[:,1,5],synthetic_experimental_data[:,3,5], c=:navy, label="E = $(synthetic_experimental_data[1,2,5]) mM")
    xlabel!("Substrate S (mM)", fontsize=18)
    ylabel!("Rate r (mM/min)", fontsize = 18);
end

LoadError: UndefVarError: `synthetic_experimental_data` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Expected value of parameters

In [12]:
(μ̂,K̂,β,y,X,E) = let

    enzyme_concentration_index = 10; # which case will we use?
    data = synthetic_experimental_data[:,:,enzyme_concentration_index];
    E = synthetic_experimental_data[1,2,enzyme_concentration_index];
    (number_of_rows, number_of_cols) = size(data);

    Y = zeros(number_of_rows); # output array
    X = ones(number_of_rows,2); # data array
    for i ∈ 1:number_of_rows
        Y[i] = 1/data[i,3]; # 1/r value
        X[i,2] = 1/data[i,1]; # 1/S value
    end

    # compute -
    β = inv(transpose(X)*X)*transpose(X)*Y
    μ̂ = (1/(β[1]*E));
    K̂ = β[2]*μ̂*E;

    # return 
    μ̂,K̂,β,Y,X,E
end;

LoadError: UndefVarError: `synthetic_experimental_data` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [13]:
let
    df = DataFrame();
    row_df = (
        μ = μ,
        μ̂ = μ̂,
        rtol_μ = ((μ - μ̂)/μ)*100,
        K = K,
        K̂ = K̂,
        rtol_K = ((K - K̂)/K)*100
    );
    push!(df, row_df);
    pretty_table(df, tf = tf_simple)
end

LoadError: UndefVarError: `DataFrame` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

samples

In [14]:
p̂ = let

    # compute the error model -
    residual = y - X*β;
    error_model = fit_mle(Normal, residual);
    
    
    number_of_samples = 1000;
    (number_of_rows, number_of_cols) = size(X);
    β̂ = Array{Float64,2}(undef, number_of_samples, 2);
    parameters = Array{Float64,2}(undef, number_of_samples, 2);
    Z = inv(transpose(X)*X)*transpose(X);
    is_ok_to_stop = false;
    i = 1;
    while (is_ok_to_stop == false)
        
        ϵ = rand(error_model, number_of_rows); # draw from the residual distribution
        tmp = β - Z*ϵ;

        if (tmp[1] > 0)
            β̂[i,1] = tmp[1];
            β̂[i,2] = tmp[2];
            i += 1;
        end
        
        if (i > number_of_samples)
            is_ok_to_stop = true;
        end
    end
    
    # ok, convert the beta samples back to bare parameters
    for i ∈ 1:number_of_samples
        μ = 1/(β̂[i,1]*E);
        parameters[i,1] = μ;
        parameters[i,2] = β̂[i,2]*μ*E;
    end

    parameters
end;

LoadError: UndefVarError: `y` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [15]:
let

    μ̂ = mean(p̂[:,1]);
    σ₁ = std(p̂[:,1]);
    K̂ = mean(p̂[:,2]);
    σ₂ = std(p̂[:,2]);
    
    df = DataFrame();
    row_df = (
        μ = μ,
        μ̂ = μ̂,
        σ₁ = σ₁,
        rtol_μ = ((μ - μ̂)/μ)*100,
        K = K,
        K̂ = K̂,
        σ₂ = σ₂,
        rtol_K = ((K - K̂)/K)*100
    );
    push!(df, row_df);
    pretty_table(df, tf = tf_simple)
    
end

LoadError: UndefVarError: `p̂` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [16]:
β, X, y = let
    
    D = house_training; # what dataset are going to use?
    number_of_examples = size(D,1); # how many examples do we have (rows)
    X = [D[:,1:end-1] ones(number_of_examples)]; # features: need to add a 1 to each row (for bias)
    y = D[:,end]; # output: this is the target data

    (number_of_examples, number_of_features) = size(X);
    IM = Matrix(1.0*I, number_of_features, number_of_features);
    A = inv(transpose(X)*X)*transpose(X);
    β = A*y; # this is the expected value of the parameters w/o regularization

    β,X,y
end;

LoadError: UndefVarError: `house_training` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Visualize

In [17]:
let

    Y = Array{Float64,2}(undef, number_of_training_examples_housing, 2);
    for i ∈ 1:number_of_training_examples_housing
        Y[i,1] = dot(X[i,:],β);
        Y[i,2] = y[i];
    end

    XYLINE = range(0,stop=50,length=100) |> collect;
    scatter(Y[:,1], Y[:,2], c=:gray67, label="Training dataset")
    plot!(XYLINE,XYLINE,c=:red, lw=2, label="Equality", ls=:dash)
    xlabel!("Predicted target variable", fontsize=18)
    ylabel!("Observed target variable", fontsize=18)
    
end

LoadError: UndefVarError: `number_of_training_examples_housing` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Goodness of fit?

In [18]:
let
    Y = Array{Float64,2}(undef, number_of_training_examples_housing, 2);
    for i ∈ 1:number_of_training_examples_housing
        Y[i,1] = dot(X[i,:],β);
        Y[i,2] = y[i];
    end
    # ȳ = mean(Y[:,2]); # mean of the observed data
    # SS_res = sum((Y[:,2] .- Y[:,1]).^2)
    # SS_tol = sum((Y[:,2] .- ȳ).^2);
    # Rsq = 1 - SS_res/SS_tol;
    r = cor(Y[:,1], Y[:,2]) |> x-> x^2

    df = DataFrame();
    row_df = (
        Rsq = r,
        FUV = 1 - r
    )
    push!(df, row_df);
    pretty_table(df, tf=tf_simple)
end

LoadError: UndefVarError: `number_of_training_examples_housing` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

### Case II: Regularized linear regression models
Regularized linear regression models incorporate penalty terms to constrain the size of the coefficient estimates, thereby reducing overfitting and enhancing the model's generalizability to new data. Consider an overdetermined data matrix $\mathbf{X}\in\mathbb{R}^{n\times{p}}$, i.e., the case where $n>p$ (more examples than unknown parameters).

A regularized least squares estimate of the unknown parameters $\mathbf{\beta}$ for an _overdetermined_ system will _minimize_ a loss (objective) function of the form:
$$
\begin{equation*}
\hat{\mathbf{\beta}} = \arg\min_{\mathbf{\beta}} ||~\mathbf{y} - \mathbf{X}\cdot\mathbf{\beta}~||^{2}_{2} + \lambda\cdot||~\mathbf{\beta}~||^{2}_{2}
\end{equation*}
$$
where $||\star||^{2}_{2}$ is the square of the [`p = 2` vector norm](https://en.wikipedia.org/wiki/Norm_(mathematics)#Euclidean_norm), $\lambda\geq{0}$ denotes a regularization parameter, and $\hat{\mathbf{\beta}}$ denotes the estimated parameter vector. 
The parameters $\hat{\mathbf{\beta}}$ that minimize the $||\star||^{2}_{2}$ loss plus penalty for overdetermined data matrix $\mathbf{X}$ are given by:
$$
\begin{equation*}
\hat{\mathbf{\beta}}_{\lambda} = \left(\mathbf{X}^{T}\mathbf{X}+\lambda\cdot\mathbf{I}\right)^{-1}\mathbf{X}^{T}\mathbf{y} - \left(\mathbf{X}^{T}\mathbf{X}+\lambda\cdot\mathbf{I}\right)^{-1}\mathbf{X}^{T}\mathbf{\epsilon}
\end{equation*}
$$
The matrix $\mathbf{X}^{T}\mathbf{X}+\lambda\cdot\mathbf{I}$ is the $\texttt{regularized normal matrix}$, while $\mathbf{X}^{T}\mathbf{y}$ is the $\texttt{moment vector}$. The inverse $\left(\mathbf{X}^{T}\mathbf{X}+\lambda\cdot\mathbf{I}\right)^{-1}$ must exist to obtain the estimated parameter vector $\hat{\mathbf{\beta}}_{\lambda}$.

In [19]:
β̂, X, y, λ = let

    # constants and data -
    λ = 60.0; # regularization parameter λ ≥ 0 (60 is a good value)
    number_of_samples = number_of_training_examples_housing; # we'll generate number_of_training_examples random parameters
    D = house_training; # what dataset are going to use?
    number_of_examples = size(D,1); # how many examples do we have (rows)
    X = [D[:,1:end-1] ones(number_of_examples)]; # features: need to add a 1 to each row (for bias)
    y = D[:,end]; # output: this is the target data

    (number_of_examples, number_of_features) = size(X);
    IM = Matrix(1.0*I, number_of_features, number_of_features);
    A = inv(transpose(X)*X + λ*IM)*transpose(X);

    # compute the error model -
    residual = y - X*A*y;
    error_model = fit_mle(Normal, residual);

    # sample the error model -
    β̂ = Array{Float64,2}(undef, number_of_samples, number_of_features);
    for s ∈ 1:number_of_samples
        ϵ = rand(error_model, number_of_examples);
        tmp = A*y - A*ϵ;
        for j ∈ 1:number_of_features
            β̂[s,j] = tmp[j];
        end
    end

    # return the data
    β̂, X, y, λ
end;

LoadError: UndefVarError: `number_of_training_examples_housing` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [20]:
β̂

LoadError: UndefVarError: `β̂` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [21]:
let
    parameter_index = 1;
    d = fit_mle(Normal,β̂[:,parameter_index])
    density(β̂[:,parameter_index], label="Data")
    plot!(d, label="Normal distribution MLE")
end

LoadError: UndefVarError: `fit_mle` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [22]:
Y_training, Y_test = let

    # training -
    number_of_parameter_sets = size(β̂,1);
    number_of_examples = size(X,1);
    Y_training = Array{Float64,2}(undef, number_of_parameter_sets, 2);
    for i ∈ 1:number_of_parameter_sets
        Y_training[i,1] = dot(X[i,:],β̂[i,:]);
        Y_training[i,2] = y[i] # training
    end

    # testing -
    D = house_test; # what dataset are going to use?
    number_of_examples = size(D,1); # how many examples do we have (rows) in the dataset D?
    X_test = [D[:,1:end-1] ones(number_of_examples)]; # features: need to add a 1 to each row (for bias)
    y_test = D[:,end]; # output: this is the target data
    Y_test = Array{Float64,2}(undef, number_of_examples, 2);
    for i ∈ 1:number_of_examples
        Y_test[i,1] = dot(X_test[i,:],β̂[i,:]);
        Y_test[i,2] = y_test[i] # training
    end

    Y_training, Y_test
end;

LoadError: UndefVarError: `β̂` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Visualize.

In [23]:
let
    XYLINE = range(0,stop=50,length=100) |> collect;
    scatter(Y_training[:,1], Y_training[:,2], c=:gray67, label="Training dataset")
    scatter!(Y_test[:,1], Y_test[:,2], c=:black, label="Test dataset")
    plot!(XYLINE,XYLINE,c=:red, lw=2, label="Equality", ls=:dash)
    xlabel!("Predicted target variable", fontsize=18)
    ylabel!("Observed target variable", fontsize=18)
end

LoadError: UndefVarError: `Y_training` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Goodness of fit

In [24]:
let
    r_training = cor(Y_training[:,1], Y_training[:,2]) |> x-> x^2
    r_test = cor(Y_test[:,1], Y_test[:,2]) |> x-> x^2

    df = DataFrame();
    row_df = (
        λ = λ,
        Rsq_training = r_training,
        FUV_training = 1 - r_training,
        Rsq_test = r_test,
        FUV_test = 1 - r_test,
    )
    push!(df, row_df);
    pretty_table(df, tf=tf_simple)
end

LoadError: UndefVarError: `Y_training` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

### Case III: Underdetermind data matrix
Assume the data matrix $\mathbf{X}$ is $\texttt{underdetermined}$, i.e., $n < p$ (more columns than rows), and 
the error vector $\mathbf{\epsilon}\sim\mathcal{N}(\mathbf{0},\sigma^{2}\cdot\mathbf{I})$.
Then, an [ordinary least squares](https://en.wikipedia.org/wiki/Ordinary_least_squares) estimate of the unknown parameters is the $\textit{smallest}$ parameter vector $\beta$ that satisfies the original equations:
$$
\begin{eqnarray*}
\text{minimize}~& & ||\,\mathbf{\beta}\,|| \\
\text{subject to} & & \mathbf{X}\cdot\mathbf{\beta} = \mathbf{y}
\end{eqnarray*}
$$
The least-norm problem has an analytical estimate for the unknown parameter vector $\hat{\mathbf{\beta}}$ given by:
$$
\begin{equation*}
\hat{\mathbf{\beta}} =\mathbf{X}^{T}\left(\mathbf{X}\mathbf{X}^{T}\right)^{-1}\cdot\mathbf{y} - \mathbf{X}^{T}\left(\mathbf{X}\mathbf{X}^{T}\right)^{-1}\cdot\mathbf{\epsilon}
\end{equation*}
$$
where inverse $\left(\mathbf{X}\mathbf{X}^{T}\right)^{-1}$ must exist to obtain the estimated model parameter vectors $\hat{\mathbf{\beta}}$.

In [25]:
# example goes here

## Linear regression models for classification tasks
Linear regression can be adapted for classification tasks by transforming the continuous output of the linear regression model directly to a class designation, e.g., $\sigma:\mathbb{R}\rightarrow\{-1,+1\}$ or into a probability using an output function $\sigma:\mathbb{R}\rightarrow\mathbb{R}$ and applying a threshold to categorize predictions into discrete classes. Let's take a look at two examples of these strategies:

* [The Perceptron (Rosenblatt, 1957)](https://en.wikipedia.org/wiki/Perceptron) is a simple yet powerful algorithm used in machine learning for binary classification tasks. It operates by _incrementally_ learning a linear decision boundary (linear regression model) that separates two classes based on input features by directly mapping the continuous output to a class such as $\sigma:\mathbb{R}\rightarrow\{-1,+1\}$, where the output function is $\sigma(\star) = \text{sign}(\star)$.
* [Logistic regression](https://en.wikipedia.org/wiki/Logistic_regression#) is a statistical method used in machine learning for binary classification tasks using the [logistics function](https://en.wikipedia.org/wiki/Logistic_function) as the transformation function. Applying the logistic function transforms the output of a linear regression model into a probability, enabling effective decision-making in various applications. We'll consider this approach next time.

### Perceptron
[The Perceptron (Rosenblatt, 1957)](https://en.wikipedia.org/wiki/Perceptron) takes the (scalar) output of a linear regression model $y_{i}\in\mathbb{R}$ and then transforms it using the $\sigma(\star) = \text{sign}(\star)$ function to a discrete set of values representing categories, e.g., $\sigma:\mathbb{R}\rightarrow\{-1,1\}$ in the binary classification case. 
* Suppose there exists a data set.
$\mathcal{D} = \left\{(\mathbf{x}_{1},y_{1}),\dotsc,(\mathbf{x}_{m},y_{m})\right\}$ with $m$ _labeled_ examples, where each example $1,2,\dots,m$ has been labeled by an expert, i.e., a human to be in a category $\hat{y}_{i}\in\{-1,1\}$, given the feature vector $\mathbf{x}_{i}\in\mathbb{R}^{n}$. 
* The Perceptron _incrementally_ learns a linear decision boundary between _two_ classes of possible objects (binary classification) in $\mathcal{D}$ by repeatedly processing the data. During each pass, a regression parameter vector $\mathbf{\beta}$ is updated until it makes no more than a specified number of mistakes. 

The Perceptron computes the label $\hat{y}_{i}$ for feature vector $\mathbf{x}_{i}$ using the $\sigma(\star) = \text{sign}(\star)$ function:
$$
\begin{equation*}
    \hat{y}_{i} = \text{sign}\left(\mathbf{x}_{i}^{T}\cdot\beta\right)
\end{equation*}
$$
where $\beta=\left(w_{1},\dots,w_{n}, b\right)$ is a column vector of (unknown) weight parameters $w_{j}\in\mathbb{R}$ corresponding to the importance of feature $j$ and a   bias parameter $b\in\mathbb{R}$, the features $\mathbf{x}^{T}_{i}=\left(x^{(i)}_{1},\dots,x^{(i)}_{n}, 1\right)$ is the $n+1$-dimensional feature (row) vector (features augmented with bias term), and $\text{sign}(z)$ is the $\texttt{sign}$ function:
$$
\begin{equation*}
    \text{sign}(z) = 
    \begin{cases}
        1 & \text{if}~z\geq{0}\\
        -1 & \text{if}~z<0
    \end{cases}
\end{equation*}
$$
__Hypothesis__: If data set $\mathcal{D}$ is linearly separable, the Perceptron will _incrementally_ learn a separating hyperplane in a finite number of passes through the data set $\mathcal{D}$. However, if the data set $\mathcal{D}$ is not linearly separable, the Perceptron may not converge. Check out a [perceptron pseudo-code here!](https://github.com/varnerlab/CHEME-5820-Lectures-Spring-2025/blob/main/lectures/week-3/L3a/figs/pcode-perceptron.pdf)

In [26]:
model = let

    # data -
    D = banknote_training; # what dataset are going to use?
    number_of_examples = size(D,1); # how many examples do we have (rows)
    number_of_features = size(D,2); # how many features do we have (cols)?
    X = [D[:,1:end-1] ones(number_of_examples)]; # features: need to add a 1 to each row (for bias), after removing the label
    y = D[:,end]; # output: this is the target data (label)

    # model
    model = build(MyPerceptronClassificationModel, (
        parameters = ones(number_of_features),
        mistakes = 0 # willing to like with m mistakes
    ));

    # train -
    model = learn(X,y,model, maxiter = 1000, verbose = true);

    # return -
    model;
end

LoadError: UndefVarError: `banknote_training` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [27]:
ŷ,y = let

    D = banknote_test; # what dataset are going to use?
    number_of_examples = size(D,1); # how many examples do we have (rows)
    number_of_features = size(D,2); # how many features do we have (cols)?
    X = [D[:,1:end-1] ones(number_of_examples)]; # features: need to add a 1 to each row (for bias), after removing the label
    y = D[:,end]; # output: this is the *actual* target data (label)

    # compute the estimated labels -
    ŷ = classify(X,model)

    # return -
    ŷ,y
end;

LoadError: UndefVarError: `banknote_test` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

How many mistakes do the classifier make?

In [28]:
number_of_prediction_mistakes = let

    number_of_test_examples = length(ŷ);
    error_counter = 0;

    for i ∈ 1:number_of_test_examples
        if (ŷ[i] != y[i])
            error_counter += 1;
        end
    end
    
    error_counter
end;

LoadError: UndefVarError: `ŷ` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [29]:
println("Perceptron mistake percentage: $((number_of_prediction_mistakes/length(ŷ))*100)%")

LoadError: UndefVarError: `number_of_prediction_mistakes` not defined in `Main`
Suggestion: check for spelling errors or missing imports.